In [6]:
from dataclasses import dataclass
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import LabelEncoder

In [7]:
class NewsDataset(Dataset):
    def __init__(self, feature_path, label_path, label_col):

        self.features = np.load(feature_path)
        self.labels = pd.read_csv(label_path)[label_col]
        self.label_encoder = LabelEncoder()
        self.encoded_labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.encoded_labels[idx], dtype=torch.long)
        return feature, label

def get_loader(train_feature_path, train_label_path, test_feature_path, test_label_path, batch_size, valid_size, col='Stance'):
    train_valid_data = NewsDataset(train_feature_path, train_label_path, col)
    test_data = NewsDataset(test_feature_path, test_label_path, col)
    train_data, valid_data = random_split(train_valid_data, [1 - valid_size, valid_size])
    return DataLoader(train_data, batch_size = batch_size, shuffle = True), DataLoader(valid_data, batch_size = batch_size, shuffle = True), DataLoader(test_data, batch_size=batch_size, shuffle = False)

train_loader, valid_loader, test_loader = get_loader('./features/feature_train.npy', './features/label_train.csv', './features/feature_test.npy', './features/label_test.csv', batch_size=32, valid_size=0.2)

print('All data preprocessed')

All data preprocessed


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout=0.1):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_sizes[0], batch_first=True)
        self.dropout1 = nn.Dropout(dropout)

        self.lstm2 = nn.LSTM(hidden_sizes[0], hidden_sizes[1], batch_first=True)
        self.dropout2 = nn.Dropout(dropout)

        self.lstm3 = nn.LSTM(hidden_sizes[1], hidden_sizes[2], batch_first=True)
        self.dropout3 = nn.Dropout(dropout)

        self.fc = nn.Linear(hidden_sizes[2], output_size)

    def forward(self, x):
        x = x.unsqueeze(1)
        x, _ = self.lstm1(x)
        x = self.dropout1(x)

        x, _ = self.lstm2(x)
        x = self.dropout2(x)

        x, _ = self.lstm3(x)
        x = self.dropout3(x)
        x = x.flatten(start_dim=1)
        x = self.fc(x)
        return x

input_size = train_loader.dataset[0][0].shape[0]
hidden_sizes = [64, 32, 16]
output_size = 4

model = LSTMModel(input_size, hidden_sizes, output_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

def train_model(model, train_loader, valid_loader, num_epochs=500):
    train_losses, val_losses = [], []
    best_val_loss = float('inf')
    patience, patience_counter = 20, 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()
        
        train_losses.append(train_loss / len(train_loader))
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in valid_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                y_pred = model(X_batch)
                val_loss += criterion(y_pred, y_batch).item()
        val_loss /= len(valid_loader)
        val_losses.append(val_loss)
        scheduler.step(val_loss)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_loss:.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                model.load_state_dict(torch.load('best_model.pth'))
                break
    
    return train_losses, val_losses

train_losses, val_losses = train_model(model, train_loader, valid_loader, num_epochs=500)

model.eval()
y_pred_list, y_true_list = [], []